In [34]:
from pseudoproof.ml_logic.model import gbc_model
# from pseudoproof.ml_logic.preproc import *
from pseudoproof.ml_logic.preproc import clean_data,scale_data,digit_freq
import pandas as pd
import joblib

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from keras import Model, Sequential, optimizers
from keras.layers import Dense
from keras.callbacks import EarlyStopping

contents = pd.read_csv("/Users/anaispepey/code/APepey/PseudoProof/raw_data/datasets/complete_dataset_true_fake.csv")


## working workflow:
### to do: retrain models on final training dataset
### add same structure for RNN


## Data

In [9]:
df = clean_data(contents)

X = df.drop(columns='y')
y = df["y"]

X_scaled = scale_data(X)
X_final = digit_freq(X_scaled)

# split

# X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_split=0.3)

X_train, X_test = X_final[:-1], X_final[-1:]
y_train, y_test = y[:-1], y[-1:]


✅ data cleaned of NAs and duplicates
✅ data scaled
✅ digits frequency computed 
final shape: (374, 20)


In [10]:
X_train.sample()


,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,s_0,s_1,s_2,s_3,s_4,s_5,s_6,s_7,s_8,s_9
311,0.05,0.0,0.0,0.15,0.0,0.05,0.0,0.0,0.0,0.05,0.0,0.0,0.0,0.05,0.05,0.05,0.0,0.0,0.05,0.1


## Models list

# First 5 Models

In [12]:
#parameters to be updated when gridsearch is complete
knn_model = KNeighborsClassifier(n_neighbors=5)
nb_model = GaussianNB()
gbc_model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1)
svm_model = SVC(kernel="rbf", C=1, random_state=42)
rf_model = RandomForestClassifier(n_estimators=100)


In [13]:
knn_model = knn_model.fit(X_train, y_train)
nb_model = nb_model.fit(X_train, y_train)
gbc_model = gbc_model.fit(X_train, y_train)
svm_model = svm_model.fit(X_train, y_train)
rf_model = rf_model.fit(X_train, y_train)


In [15]:

MODELS = {
    knn_model: "knn",
    nb_model: "naive_bayes",
    gbc_model: "gradient_boosting",
    rf_model: "random_forest",
    svm_model: "svm",
}


In [16]:
key_value_pairs = MODELS.items()

for key, value in key_value_pairs:
    model_path = f"/Users/anaispepey/code/APepey/PseudoProof/trained_models/{value}.pkl"
    joblib.dump(key, filename=model_path)


#last step: drag all <model>.pkl files to pseudoproof Google Bucket


# RNN model - to do

#train RNN model
#save it in trained_models
#drag it to google bucket 'pseudoproof'
#load it into local machine

In [38]:
#itinialize
input_shape = X_train.shape

rnn_model = Sequential()
rnn_model.add(Dense(10, input_shape=input_shape, activation="relu"))
rnn_model.add(Dense(20, activation="relu"))
rnn_model.add(Dense(15, activation="relu"))
rnn_model.add(Dense(8, activation="relu"))
rnn_model.add(Dense(1, activation="sigmoid"))

#compile
learning_rate = 0.001

optimizer = optimizers.Adam(learning_rate=learning_rate)
rnn_model.compile(
        loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"]
    )

#train
patience=2
validation_split=0.3
epochs=100
batch_size=256

es = EarlyStopping(
        monitor="val_loss", patience=patience, restore_best_weights=True, verbose=1
    )

history = rnn_model.fit(
        X_train,
        y_train,
        # validation_data=validation_data,
        validation_split=validation_split,
        batch_size=batch_size,
        epochs = epochs,
        callbacks=[es],
        verbose=0,
    )

print(history)


ValueError: in user code:

    File "/Users/anaispepey/.pyenv/versions/3.10.6/envs/PseudoProof/lib/python3.10/site-packages/keras/engine/training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "/Users/anaispepey/.pyenv/versions/3.10.6/envs/PseudoProof/lib/python3.10/site-packages/keras/engine/training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/anaispepey/.pyenv/versions/3.10.6/envs/PseudoProof/lib/python3.10/site-packages/keras/engine/training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "/Users/anaispepey/.pyenv/versions/3.10.6/envs/PseudoProof/lib/python3.10/site-packages/keras/engine/training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "/Users/anaispepey/.pyenv/versions/3.10.6/envs/PseudoProof/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        

In [ ]:
MODELS[rnn_model] = "rnn"


NameError: name 'rnn_model' is not defined